In [1]:
# #Import rapids library,
# import sys
# !cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#import cudf
#import cupy as cp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rapids/rapids.0.15.0
/kaggle/input/rapids/rapids.0.14.0
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv


In [3]:
#cudf.__version__

In [4]:
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from cuml.linear_model import LogisticRegression

In [5]:
train_features=pd.read_csv('../input/lish-moa/train_features.csv')
test_features=pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
sample_submission=pd.read_csv('../input/lish-moa/sample_submission.csv')

In [6]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [7]:
targets=train_targets_scored.columns[1:]
len(targets)

206

In [8]:
ctrl_vehicle_idx=test_features[test_features['cp_type']=='ctl_vehicle']['sig_id'].values
len(ctrl_vehicle_idx)

358

In [9]:
tr_idx=train_features[train_features['cp_type']!='ctl_vehicle']['sig_id']
te_idx=test_features[test_features['cp_type']!='ctl_vehicle']['sig_id']

In [10]:
##train features,

#test_df=test_features.loc[te_idx,:]
train_df=pd.concat([train_features,pd.get_dummies(train_features['cp_time'],prefix='cp_time')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_type'],prefix='cp_type')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_dose'],prefix='cp_dose')],axis=1)
train_df=train_df.drop(['cp_time','cp_type','cp_dose'],axis=1)

#test features,
test_df=pd.concat([test_features,pd.get_dummies(test_features['cp_time'],prefix='cp_time')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_type'],prefix='cp_type')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_dose'],prefix='cp_dose')],axis=1)
test_df=test_df.drop(['cp_time','cp_type','cp_dose'],axis=1)

In [11]:
train_features.shape,test_features.shape

((23814, 876), (3982, 876))

In [12]:
train_df.shape,test_df.shape

((23814, 880), (3982, 880))

In [13]:
train_df.drop('sig_id',axis=1,inplace=True)
test_df.drop('sig_id',axis=1,inplace=True)

In [14]:
kfolds=StratifiedKFold(n_splits=3,random_state=42,shuffle=True)

In [15]:
tar=['5-alpha_reductase_inhibitor','atp-sensitive_potassium_channel_antagonist','11-beta-hsd1_inhibitor']

In [16]:
feats=[c for c in train_features.columns if c not in ['sig_id']]

In [17]:
def log_loss_metric(y_true,y_preds):
    """
    Custom Log loss
    """
    y_pred_clip=np.clip(y_preds,1e-15,1-1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip)))
    return loss

In [18]:
#sel=VarianceThreshold(threshold=1.5).fit(train_features[feats])

In [19]:
# train_df=sel.transform(train_features[feats])
# test_df=sel.transform(test_features[feats])


In [20]:
#train_targets_scored.shape,train_targets_scored.loc[tr_idx,:].shape

In [21]:
# scores=[]
# non_pred_class=[]
# for class_name in tar:
#     #class_preds=np.zeros(len(train_targets_scored))
#     train_targets=train_targets_scored[class_name]
#     if train_targets.sum()>5:
#         classifier=LogisticRegression(C=2,solver='saga',n_jobs=-1)
#         cv_score=-np.mean(cross_val_score(classifier,train_df,train_targets,cv=3,scoring='neg_log_loss',n_jobs=-1))
#         scores.append(cv_score)
#         print(f'CV Score for class {class_name} is {cv_score}')
        
#         classifier.fit(train_df,train_targets)
#         sample_submission[class_name]=classifier.predict_proba(test_df)[:,1]
#     else:
#         non_pred_class.append(class_name)
    
# print(f'Total CV Score is {np.mean(scores)}')

In [22]:
scores=[]
non_pred_class=[]
for class_name in targets:
    #class_preds=np.zeros(len(train_targets_scored))
    train_targets=train_targets_scored[class_name]
    if train_targets.sum()>5:
        classifier=RandomForestClassifier(n_estimators=80,
                                         max_depth=10,
                                         min_samples_split=10,
                                        min_samples_leaf=10,
                                          max_features='sqrt',
                                         n_jobs=-1,
                                         random_state=42
                                         )
        cv_score=-np.mean(cross_val_score(classifier,train_df,train_targets,cv=3,scoring='neg_log_loss',n_jobs=-1))
        scores.append(cv_score)
        print(f'CV Score for class {class_name} is {cv_score}')
        
        classifier.fit(train_df,train_targets)
        sample_submission[class_name]=classifier.predict_proba(test_df)[:,1]
    else:
        non_pred_class.append(class_name)
    
print(f'Total CV Score is {np.mean(scores)}')

CV Score for class 5-alpha_reductase_inhibitor is 0.015801082670174247
CV Score for class 11-beta-hsd1_inhibitor is 0.017638171749466604
CV Score for class acat_inhibitor is 0.013376687040160951
CV Score for class acetylcholine_receptor_agonist is 0.04805579224226402
CV Score for class acetylcholine_receptor_antagonist is 0.06797743731930975
CV Score for class acetylcholinesterase_inhibitor is 0.02258220889999844
CV Score for class adenosine_receptor_agonist is 0.01781902289776865
CV Score for class adenosine_receptor_antagonist is 0.028200093033047966
CV Score for class adenylyl_cyclase_activator is 0.008196076364063415
CV Score for class adrenergic_receptor_agonist is 0.06211252285862156
CV Score for class adrenergic_receptor_antagonist is 0.0781515637079268
CV Score for class akt_inhibitor is 0.017019271209349183
CV Score for class aldehyde_dehydrogenase_inhibitor is 0.0018688173874662883
CV Score for class alk_inhibitor is 0.013001748481713397
CV Score for class ampk_activator is 0

In [23]:
sample_submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000006,0.000003,0.000065,0.018274,0.018840,0.001251,0.000470,0.002358,0.0,...,0.0,0.000056,0.000640,0.001228,0.001076,0.0,0.002655,0.000809,0.000136,0.000316
1,id_001897cda,0.000000,0.000011,0.000069,0.004360,0.010994,0.001406,0.000309,0.003673,0.0,...,0.0,0.000030,0.003664,0.005826,0.000990,0.0,0.013837,0.000050,0.000092,0.000766
2,id_002429b5b,0.000008,0.004471,0.000056,0.007220,0.008143,0.001092,0.003148,0.001955,0.0,...,0.0,0.000037,0.000576,0.006064,0.005810,0.0,0.008272,0.002733,0.000109,0.000238
3,id_00276f245,0.000002,0.000006,0.000049,0.006830,0.008610,0.001166,0.000493,0.003765,0.0,...,0.0,0.000061,0.002057,0.001330,0.001255,0.0,0.005737,0.005035,0.000107,0.000225
4,id_0027f1083,0.000002,0.000010,0.000092,0.006639,0.009615,0.001360,0.002648,0.004870,0.0,...,0.0,0.000663,0.000631,0.011350,0.000863,0.0,0.001398,0.001598,0.000075,0.000191


In [24]:
sample_submission.loc[:,non_pred_class]=0
sample_submission.loc[sample_submission['sig_id'].isin(ctrl_vehicle_idx),1:]=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  


In [25]:
sample_submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000006,0.000003,0.000065,0.018274,0.018840,0.001251,0.000470,0.002358,0.0,...,0.0,0.000056,0.000640,0.001228,0.001076,0.0,0.002655,0.000809,0.000136,0.000316
1,id_001897cda,0.000000,0.000011,0.000069,0.004360,0.010994,0.001406,0.000309,0.003673,0.0,...,0.0,0.000030,0.003664,0.005826,0.000990,0.0,0.013837,0.000050,0.000092,0.000766
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000002,0.000006,0.000049,0.006830,0.008610,0.001166,0.000493,0.003765,0.0,...,0.0,0.000061,0.002057,0.001330,0.001255,0.0,0.005737,0.005035,0.000107,0.000225
4,id_0027f1083,0.000002,0.000010,0.000092,0.006639,0.009615,0.001360,0.002648,0.004870,0.0,...,0.0,0.000663,0.000631,0.011350,0.000863,0.0,0.001398,0.001598,0.000075,0.000191


In [26]:
#feature_importance_df.to_csv('feature_importance.csv',index=False)
sample_submission.to_csv('submission.csv',index=False)